# Full Data Calculation for Weapons

## Setting Up

In [1]:
import pandas as pd
from scripts.weapon import max_shots, burst
from scripts.accuracy import average_to_hit

aliens = pd.read_csv("datasets/aliens.csv")
weapon_stats = pd.read_csv("datasets/weapon_stats.csv")
ammo_stats = pd.read_csv("datasets/ammo_stats.csv")
weapon_firemodes = pd.read_csv("datasets/weapon_firemodes.csv")
damage_modifiers = pd.read_csv("datasets/damage_modifiers.csv")

## Data Merge

In [2]:
weapon_data = weapon_stats.merge(ammo_stats, on="weapon")
weapon_data = weapon_data.merge(weapon_firemodes, on="weapon")

## Chance to Hit

We will use an average Accuracy of 50. Meaning the soldier fails half of the time.

The chance of hit formula is simple. Just multiply the soldier's accuracy chance by the weapon's accuracy chance. So an average of 50 to a weapon of Accuracy 90 means 0.50 * 0.9 = 0.45 chance of hit.

In [3]:
weapon_data["average_chance_to_hit"] = weapon_data.apply(lambda x: average_to_hit(x["accuracy"]), axis=1)

## Shots

In [4]:
weapon_data["burst"] = weapon_data.apply(burst, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turn_max_shots"] = weapon_data.apply(max_shots, axis=1)
weapon_data["turns_to_reload"] = weapon_data.apply(lambda x: x["capacity"] / x["turn_max_shots"], axis=1)

## Damage Range

In [5]:
weapon_data["turn_damage_max"] = weapon_data.apply(lambda x: x["turn_max_shots"] * x["damage"], axis=1)
weapon_data["turn_damage_expected"] = weapon_data.apply(lambda x: x["turn_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data["clip_damage"] = weapon_data.apply(lambda x: x["capacity"] * x["damage"], axis=1)

## Costs

In [6]:
weapon_data["damage_per_time_unit"] = weapon_data.apply(lambda x: x["damage"] / x["time_units"], axis=1)

## Damage to Target Type

### Data Merge

In [7]:
weapon_data_targets = weapon_data.append([weapon_data] * (len(damage_modifiers.index)-1), ignore_index=True)
weapon_data_targets["key"] = weapon_data_targets.groupby(["weapon", "ammo", "fire_mode"]).cumcount()

targets = damage_modifiers["target"]

weapon_data_targets = pd.merge(weapon_data_targets, targets, left_on="key", right_index=True, how="left", sort=False)
weapon_data_targets = weapon_data_targets.drop(columns=["key"])

### Damage vs Type

In [8]:
def damage_modifier(group):
    """
    Returns the damage modifier for the weapon target.
    """
    target = damage_modifiers[damage_modifiers["target"] == group["target"]]
    damage = target[group["damage_type"]].values[0]
    return damage / 100

In [9]:
weapon_data_targets["target_damage"] = weapon_data_targets.apply(lambda x: x["damage"] * damage_modifier(x), axis=1)
weapon_data_targets["target_turn_damage_max"] = weapon_data_targets.apply(lambda x: x["turn_max_shots"] * x["target_damage"], axis=1)
weapon_data_targets["target_turn_damage_expected"] = weapon_data_targets.apply(lambda x: x["target_turn_damage_max"] * x["average_chance_to_hit"], axis=1)

### Costs

In [10]:
weapon_data_targets["target_damage_per_time_unit"] = weapon_data_targets.apply(lambda x: x["target_damage"] / x["time_units"], axis=1)

## Damage to Alien

### Average Armor

In [11]:
aliens_armor = aliens[["alien", "armor_front", "armor_side", "armor_rear", "armor_under"]]
aliens_armor = aliens_armor.set_index("alien")
aliens_armor = aliens_armor.mean(axis=1)
aliens_armor = aliens_armor.reset_index()
aliens_armor = aliens_armor.rename(columns={0: "armor"})

### Base data

In [12]:
weapon_damages_targets = weapon_data_targets.groupby(["weapon", "target"]).mean()
weapon_damages_targets = weapon_damages_targets.reset_index()

In [13]:
weapon_data_aliens = aliens.merge(aliens_armor, on="alien")
weapon_data_aliens = weapon_data_aliens.merge(weapon_damages_targets, on="target")
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_x": "time_units"})
weapon_data_aliens = weapon_data_aliens.rename(columns={"time_units_y": "weapon_time_units"})

### Penetrating Damage

In [14]:
def penetrating_damage(base_damage, armor):
    """
    Returns the damage after applying armor.
    """
    if armor >= base_damage:
        damage = 0
    else:
        damage = base_damage - armor
    
    return damage

In [15]:
weapon_data_aliens["penetrating_damage"] = weapon_data_aliens.apply(lambda x: penetrating_damage(x["target_damage"],x["armor"]), axis=1)
weapon_data_aliens["penetrating_damage_max"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] * x["penetrating_damage"], axis=1)
weapon_data_aliens["penetrating_damage_expected"] = weapon_data_aliens.apply(lambda x: x["penetrating_damage_max"] * x["average_chance_to_hit"], axis=1)
weapon_data_aliens["penetrating_damage_crit"] = weapon_data_aliens.apply(lambda x: penetrating_damage(x["target_damage"] * 2,x["armor"]), axis=1)
weapon_data_aliens["penetrating_damage_crit_expected"] = weapon_data_aliens.apply(lambda x: x["penetrating_damage_crit"] * x["average_chance_to_hit"], axis=1)

### Hits to Kill

In [16]:
def hits_to_kill(damage, health):
    """
    Returns the number of hits it takes to kill the target.
    """
    if damage > 0:
        hits = health / damage
    else:
        hits = 1000
    
    return hits

In [17]:
weapon_data_aliens["hits_to_kill"] = weapon_data_aliens.apply(lambda x: hits_to_kill(x["penetrating_damage"], x["health"]), axis=1)
weapon_data_aliens["hits_to_kill_crit"] = weapon_data_aliens.apply(lambda x: hits_to_kill(x["penetrating_damage_crit"], x["health"]), axis=1)
weapon_data_aliens["expected_kills"] = weapon_data_aliens.apply(lambda x: x["turn_max_shots"] / x["hits_to_kill"], axis=1)

### Costs

In [18]:
weapon_data_aliens["penetrating_damage_per_time_unit"] = weapon_data_aliens.apply(lambda x: x["penetrating_damage"] / x["weapon_time_units"], axis=1)

## Data Generated

In [19]:
weapon_data.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,burst,turn_max_shots,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit
0,pistol,1,pistol_clip,26,armor_piercing,12.0,snapshot,18,60,0.3300,1,5,2.400000,130,42.900,312.0,1.444444
1,pistol,1,pistol_clip,26,armor_piercing,12.0,aimed,30,78,0.4290,1,3,4.000000,78,33.462,312.0,0.866667
2,rifle,2,rifle_clip,30,armor_piercing,20.0,auto,35,35,0.1925,3,6,3.333333,180,34.650,600.0,0.857143
3,rifle,2,rifle_clip,30,armor_piercing,20.0,snapshot,25,60,0.3300,1,4,5.000000,120,39.600,600.0,1.200000
4,rifle,2,rifle_clip,30,armor_piercing,20.0,aimed,80,110,0.6050,1,1,20.000000,30,18.150,600.0,0.375000


In [20]:
weapon_data_targets.head()

,weapon,grip,ammo,damage,damage_type,capacity,fire_mode,time_units,accuracy,average_chance_to_hit,...,turns_to_reload,turn_damage_max,turn_damage_expected,clip_damage,damage_per_time_unit,target,target_damage,target_turn_damage_max,target_turn_damage_expected,target_damage_per_time_unit
0,pistol,1,pistol_clip,26,armor_piercing,12.0,snapshot,18,60,0.3300,...,2.400000,130,42.900,312.0,1.444444,terrain,26.0,130.0,42.900,1.444444
1,pistol,1,pistol_clip,26,armor_piercing,12.0,aimed,30,78,0.4290,...,4.000000,78,33.462,312.0,0.866667,terrain,26.0,78.0,33.462,0.866667
2,rifle,2,rifle_clip,30,armor_piercing,20.0,auto,35,35,0.1925,...,3.333333,180,34.650,600.0,0.857143,terrain,30.0,180.0,34.650,0.857143
3,rifle,2,rifle_clip,30,armor_piercing,20.0,snapshot,25,60,0.3300,...,5.000000,120,39.600,600.0,1.200000,terrain,30.0,120.0,39.600,1.200000
4,rifle,2,rifle_clip,30,armor_piercing,20.0,aimed,80,110,0.6050,...,20.000000,30,18.150,600.0,0.375000,terrain,30.0,30.0,18.150,0.375000


In [21]:
weapon_data_aliens.head()

,alien,time_units,stamina,health,bravery,reactions,firing_accuracy,throwing_accuracy,strength,psionic_strength,...,target_damage_per_time_unit,penetrating_damage,penetrating_damage_max,penetrating_damage_expected,penetrating_damage_crit,penetrating_damage_crit_expected,hits_to_kill,hits_to_kill_crit,expected_kills,penetrating_damage_per_time_unit
0,Floater Soldier,50,90,35,80,50,50,58,40,35,...,1.200000,82.500000,82.500000,29.493750,172.500000,61.668750,0.424242,0.202899,2.357143,1.100000
1,Floater Soldier,50,90,35,80,50,50,58,40,35,...,1.009512,37.166667,123.888889,38.612037,81.833333,25.504722,0.941704,0.427699,3.539683,0.728758
2,Floater Soldier,50,90,35,80,50,50,58,40,35,...,2.500000,192.500000,192.500000,127.050000,392.500000,259.050000,0.181818,0.089172,5.500000,2.406250
3,Floater Soldier,50,90,35,80,50,50,58,40,35,...,1.284091,52.500000,105.000000,43.312500,112.500000,46.406250,0.666667,0.311111,3.000000,0.929204
4,Floater Soldier,50,90,35,80,50,50,58,40,35,...,2.121212,132.500000,132.500000,72.875000,272.500000,149.875000,0.264151,0.128440,3.785714,2.007576
